In [10]:
from nltk.tokenize import word_tokenize
import re
from unidecode import unidecode
import pandas as pd
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from collections import Counter
from FuzzyTM import FLSA_W
import pickle

In [12]:
file_path = 'preprocessed_data_lda_flsa.pkl'

# Read the data from the file
with open(file_path, 'rb') as file:
    prep_data = pickle.load(file)

In [ ]:
def filter_word_from_corpus(data, words):
    """
    Filters out specific words from the corpus.
    Parameters:
    - data : The corpus, represented by a list of list of tokens.
    - words : The words to be filtered. Either a list of tokenized words or a single word.
    Returns:
    - List[List[str]]: Filtered data where specified words have been removed.
    Example:
    >> data = [["apple", "orange"], ["apple", "banana"]]
    >> filter_data(data, "apple")
    [["orange"], ["banana"]]
    """
    # Ensure words is a list, even if a single string is passed
    if isinstance(words, str):
        words = [words]
    # Filter words from data
    filtered_data = [[token for token in row if token not in words] for row in data]
    return filtered_data

In [13]:
flsaw = FLSA_W(input_file = prep_data,
    num_topics = 10,
    num_words = 10,
    )

In [14]:
flsaw.get_vocabulary_size()
pwgt, ptgd = flsaw.get_matrices()
print(flsaw.show_topics())


for topic in flsaw.show_topics(representation='words'):
    print(topic)
    print(flsaw.get_coherence_score())
    print(flsaw.get_diversity_score())
    print(flsaw.get_interpretability_score())

[(0, '0.0005*"premarket" + 0.0005*"kuo" + 0.0005*"midday" + 0.0004*"chi" + 0.0004*"frankfurt" + 0.0004*"lite" + 0.0004*"forming" + 0.0004*"div" + 0.0004*"consolidation" + 0.0004*"tweeted"'), (1, '0.0082*"aapl" + 0.0072*"price" + 0.0071*"sale" + 0.0071*"share" + 0.0068*"report" + 0.0068*"product" + 0.0067*"million" + 0.0065*"service" + 0.0065*"next" + 0.0065*"inc"'), (2, '0.0022*"previously" + 0.0014*"planned" + 0.0014*"dec" + 0.0013*"cnbc" + 0.0013*"asian" + 0.0012*"yesterday" + 0.0012*"dispute" + 0.0012*"jones" + 0.0012*"lawsuit" + 0.0012*"donald"'), (3, '0.0008*"otc" + 0.0008*"thomson" + 0.0008*"taiwan" + 0.0008*"journal" + 0.0007*"aug" + 0.0007*"goog" + 0.0007*"finished" + 0.0007*"unveil" + 0.0007*"carl" + 0.0007*"ssnlf"'), (4, '0.0022*"version" + 0.0022*"york" + 0.0022*"chip" + 0.0021*"program" + 0.0021*"purchase" + 0.0021*"smartphones" + 0.002*"rival" + 0.002*"manufacturer" + 0.002*"larger" + 0.002*"computer"'), (5, '0.0003*"sma" + 0.0003*"fibonacci" + 0.0003*"kgi" + 0.0003*"conso

KeyboardInterrupt: 

In [ ]:
#After training this model, remove the useless words

### Ideeën:
- filter alle hulpwerkwoorden en bijwoorden eruit --> de stopwords zijn alleen nog maar de bezittelijke voornaamwoorden, en een deel van de hulpwerkwoorden

In [ ]:
useless_words = ['whether', 'much', 'sharply']
prep_data_it2 = filter_word_from_corpus(prep_data, useless_words)

In [ ]:
flsaw2 = FLSA_W(input_file = prep_data_it2,
    num_topics = 10,
    num_words = 10,
    )

In [ ]:
flsaw2.get_vocabulary_size()
pwgt, ptgd = flsaw2.get_matrices()
print(flsaw2.show_topics())


for topic in flsaw2.show_topics(representation='words'):
    print(topic)
    print(flsaw2.get_coherence_score())
    print(flsaw2.get_diversity_score())
    print(flsaw2.get_interpretability_score())

In [ ]:
#filtering words:
#randewich is a journalist, jimmy not much saying
useless_words_it3 = ['randewich', 'eddy', 'unnamed', 'next']
prep_data_it3 = filter_word_from_corpus(prep_data, useless_words_it3)
flsaw3 = FLSA_W(input_file=prep_data_it3,
                num_topics=10,
                num_words=10,
                )
flsaw3.get_vocabulary_size()
pwgt, ptgd = flsaw3.get_matrices()
print(flsaw3.show_topics())

for topic in flsaw3.show_topics(representation='words'):
    print(topic)
    print(flsaw3.get_coherence_score())
    print(flsaw3.get_diversity_score())
    print(flsaw3.get_interpretability_score())